In [84]:
import pandas as pd
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [2]:
ACCESS = 'ACCESS'
ACCESS_S = 'ACCESS_S'
TOKEN = 'TOKEN'
TOKEN_S = 'TOKEN_S'

In [68]:
auth = tweepy.OAuthHandler(ACCESS, ACCESS_S)
auth.set_access_token(TOKEN, TOKEN_S)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [100]:
def filterTweet(tweet, key_outside, key_inside=None):
  if key_inside is not None:
    result1 = tweet._json[str(key_outside)][str(key_inside)]
    return result1
  else:
    result2 = tweet._json[str(key_outside)]
    return result2

def replaceToNan(list_):
  replaced = []
  for i in range(len(list_)):
    new = [np.nan if val=='' else val for val in list_[i]]
    replaced.append(new)
  return replaced

def searchTweet(query, since, until, items):
  inside_user = ['id', 'name', 'screen_name', 'location', 'created_at', 'followers_count', 'friends_count', 'verified']
  inside_tweet = ['id', 'created_at', 'full_text']

  user_out = []
  tweet_out = []
  for tweet in tweepy.Cursor(api.search, q=query, since=since, until=until, tweet_mode='extended').items(items):
    out_user = []
    out_tweet = []
    
    for col_user in inside_user:
      result_user = filterTweet(tweet, 'user', col_user)
      out_user.append(result_user)
    
    for tw in inside_tweet:
      result_tw = filterTweet(tweet, tw)
      out_tweet.append(result_tw)
      
    user_out.append(out_user)
    tweet_out.append(out_tweet)
  final_user = replaceToNan(user_out)
  final_tweet = replaceToNan(tweet_out)
  return final_user, final_tweet

In [69]:
SEARCH = 'Rachel+Vennya'
SINCE = '2021-01-28'
UNTIL = '2021-02-05'
user, tweet = searchTweet(SEARCH, SINCE, UNTIL, 1000)

In [71]:
df_user = pd.DataFrame(user, columns=['id', 'name', 'screen_name', 'location', 'created_at', 'followers_count', 'friends_count', 'verified'])
df_tweet = pd.DataFrame(tweet, columns=['id', 'created_at', 'full_text'])

In [76]:
df_user

,id,name,screen_name,location,created_at,followers_count,friends_count,verified
0,892670807121330177,Ferdy Agusman,gusmanerdy,Medan,Wed Aug 02 08:58:00 +0000 2017,6,14,False
1,277434635,Karina angelika,karigel13,between his lungs,Tue Apr 05 11:03:15 +0000 2011,417,378,False
2,544170238,Kanjeng Sultan,ayiekkresna,"Kota Surabaya, Jawa Timur",Tue Apr 03 10:42:42 +0000 2012,590,333,False
3,1356180001888976899,UrExtacy,6hndrd6ty6,Bikini Bottom,Mon Feb 01 09:58:22 +0000 2021,2,114,False
4,1168873359741874176,bela,twingkerbelll,NaN,Tue Sep 03 13:08:38 +0000 2019,42,48,False
...,...,...,...,...,...,...,...,...
995,386421643,Iyeeeeeee,ampunbang_jago,MINORITAS,Fri Oct 07 08:05:30 +0000 2011,1133,139,False
996,843037086663950336,urgirlfriend,ikaaaprtw,Indonesia,Sat Mar 18 09:51:19 +0000 2017,14,37,False
997,1250051825106968576,Logout 🤸,sofarumah,iye iye ini soang 98,Tue Apr 14 13:22:45 +0000 2020,52,261,False
998,378548179,151cm,callmegece,Indonesia,Fri Sep 23 11:53:22 +0000 2011,519,508,False


In [103]:
df_user

,id,name,screen_name,location,created_at,followers_count,friends_count,verified
0,892670807121330177,Ferdy Agusman,gusmanerdy,Medan,Wed Aug 02 08:58:00 +0000 2017,6,14,False
1,277434635,Karina angelika,karigel13,between his lungs,Tue Apr 05 11:03:15 +0000 2011,417,378,False
2,544170238,Kanjeng Sultan,ayiekkresna,"Kota Surabaya, Jawa Timur",Tue Apr 03 10:42:42 +0000 2012,590,333,False
3,1356180001888976899,UrExtacy,6hndrd6ty6,Bikini Bottom,Mon Feb 01 09:58:22 +0000 2021,2,114,False
4,1168873359741874176,bela,twingkerbelll,NaN,Tue Sep 03 13:08:38 +0000 2019,42,48,False
...,...,...,...,...,...,...,...,...
995,386421643,Iyeeeeeee,ampunbang_jago,MINORITAS,Fri Oct 07 08:05:30 +0000 2011,1133,139,False
996,843037086663950336,urgirlfriend,ikaaaprtw,Indonesia,Sat Mar 18 09:51:19 +0000 2017,14,37,False
997,1250051825106968576,Logout 🤸,sofarumah,iye iye ini soang 98,Tue Apr 14 13:22:45 +0000 2020,52,261,False
998,378548179,151cm,callmegece,Indonesia,Fri Sep 23 11:53:22 +0000 2011,519,508,False


In [83]:
df_tweet

,id,created_at,full_text
0,1357479018472239104,Thu Feb 04 23:59:53 +0000 2021,"RT @mhmdadipati: @farisandani Dari orang"" yang..."
1,1357478833725845507,Thu Feb 04 23:59:09 +0000 2021,RT @iirman_: Dari Rachel Vennya jd inget kata ...
2,1357478819456720896,Thu Feb 04 23:59:05 +0000 2021,RT @handokotjung: Berhasil membuat banyak neti...
3,1357478818701733888,Thu Feb 04 23:59:05 +0000 2021,RT @yeontinnie: dari rachel vennya kita belaja...
4,1357478707884032001,Thu Feb 04 23:58:39 +0000 2021,RT @mncveronica: Cerita Rachel Vennya dan Okin...
...,...,...,...
995,1357382035023663105,Thu Feb 04 17:34:30 +0000 2021,RT @mncveronica: Cerita Rachel Vennya dan Okin...
996,1357381971668660227,Thu Feb 04 17:34:15 +0000 2021,"RT @RezkaR29: Dari okin kita belajar, cowok ka..."
997,1357381959400394753,Thu Feb 04 17:34:12 +0000 2021,RT @WiraBadarrsha: dari rachel natasya kita be...
998,1357381898998194176,Thu Feb 04 17:33:58 +0000 2021,RT @bluberriela: seeing what Rachel Vennya wen...


In [105]:
df_tweet.to_sql('Tweet2', sql_connection())